In [5]:
#######################
####BASELINE MODEL#####
#######################

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from pathlib import Path
import sys
sys.path.append(str(Path().absolute().parent))  # 添加父目录到 Python 路径
import warnings
warnings.filterwarnings('ignore')

# Read the CSV file
df = pd.read_csv('../data/bls_unemployment_by_age_group.csv')
print(df.head())

# Convert Period to numeric month
df['Month'] = df['Period'].str[1:].astype(int)

# Create a datetime column
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01')

# Create separate models for each age group
age_groups = df['Age Group'].unique()
plt.figure(figsize=(15, 10))

# Dictionary to store predictions
predictions_by_age = {}

for age_group in age_groups:
    # Filter data for this age group
    age_data = df[df['Age Group'] == age_group].copy()
    
    # Prepare data for linear regression
    X = (age_data['Year'] + age_data['Month']/12).values.reshape(-1, 1)
    y = age_data['Unemployment_Rate'].values
    
    # Create and fit the model
    model = LinearRegression()
    model.fit(X, y)
    
    # Generate future dates for prediction (10 years, monthly)
    last_date = age_data['Date'].max()
    future_dates = pd.date_range(start=last_date, periods=121, freq='M')[1:]  # 10 years * 12 months
    future_X = (future_dates.year + future_dates.month/12).values.reshape(-1, 1)
    
    # Make predictions
    future_predictions = model.predict(future_X)
    
    # Store predictions
    predictions_by_age[age_group] = pd.DataFrame({
        'Date': future_dates,
        'Predicted_Rate': future_predictions
    })
    
    # Plot historical data and predictions
    plt.plot(age_data['Date'], age_data['Unemployment_Rate'], 
             marker='.', linestyle='-', alpha=0.5, label=f'{age_group} Historical')
    plt.plot(future_dates, future_predictions, 
             linestyle='--', alpha=0.8, label=f'{age_group} Predicted')

plt.title('Unemployment Rate Predictions by Age Group (Next 10 Years)')
plt.xlabel('Year')
plt.ylabel('Unemployment Rate (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# Print predictions for each age group
print("\nPredicted Unemployment Rates by Age Group:")
for age_group in age_groups:
    print(f"\n{age_group}:")
    yearly_pred = predictions_by_age[age_group].set_index('Date')['Predicted_Rate']
    yearly_pred = yearly_pred.resample('Y').mean()
    print(yearly_pred.round(1)) 

FileNotFoundError: [Errno 2] No such file or directory: '../data/bls_unemployment_by_age_group.csv'

In [ ]:
#######################
####BEST MODEL#########
#######################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

#df = pd.read_csv('/content/drive/Keming'sDrive/best_model/bls_unemployment_by_age_group.csv')
df = pd.read_csv('../data/bls_unemployment_by_age_group.csv')
print(df.head())


df['Month'] = df['Period'].str[1:].astype(int)

df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01')

age_groups = df['Age Group'].unique()
plt.figure(figsize=(15, 10))


predictions_by_age = {}

for age_group in age_groups:

    age_data = df[df['Age Group'] == age_group].copy()
    X = (age_data['Year'] + age_data['Month'] / 12).values.reshape(-1, 1)
    y = age_data['Unemployment_Rate'].values

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))


    model = Sequential([
        Input(shape=(1,)),
        Dense(512, activation='relu'),
        Dense(512, activation='relu'),
        Dense(1)  # Output layer
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    model.fit(X_scaled, y_scaled, epochs=50, batch_size=8, verbose=0)

    #(10 years, monthly)
    last_date = age_data['Date'].max()
    future_dates = pd.date_range(start=last_date, periods=121, freq='M')[1:]  # 10 years * 12 months
    future_X = (future_dates.year + future_dates.month / 12).values.reshape(-1, 1)
    future_X_scaled = scaler_X.transform(future_X)

    future_predictions_scaled = model.predict(future_X_scaled)
    future_predictions = scaler_y.inverse_transform(future_predictions_scaled)
    predictions_by_age[age_group] = pd.DataFrame({
        'Date': future_dates,
        'Predicted_Rate': future_predictions.flatten()
    })

    #historical data and predictions
    plt.plot(age_data['Date'], age_data['Unemployment_Rate'], 
             marker='.', linestyle='-', alpha=0.5, label=f'{age_group} Historical')
    plt.plot(future_dates, future_predictions, 
             linestyle='--', alpha=0.8, label=f'{age_group} Predicted')

plt.title('Unemployment Rate Predictions by Age Group (Next 10 Years)')
plt.xlabel('Year')
plt.ylabel('Unemployment Rate (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


print("\nPredicted Unemployment Rates by Age Group:")
for age_group in age_groups:
    print(f"\n{age_group}:")
    yearly_pred = predictions_by_age[age_group].set_index('Date')['Predicted_Rate']
    yearly_pred = yearly_pred.resample('Y').mean()
    print(yearly_pred.round(1))

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# Define base population estimates for each channel (these are example numbers)
channel_population = {
    'Industry': {
        'total_pool': 1_000_000,  # Domestic workforce in relevant industry
        'distribution': {
            '16 to 19 years': 0.00,  # Almost none
            '20 to 24 years': 0.10,  # 10% of industry workforce
            '25 to 34 years': 0.35,  # 35% of industry workforce
            '35 to 44 years': 0.30,  # 30% of industry workforce
            '45 to 54 years': 0.15,  # 15% of industry workforce
            '55 years and over': 0.10  # 10% of industry workforce
        }
    },
    'Student': {
        'total_pool': 200_000,  # Annual graduates in relevant fields
        'distribution': {
            '16 to 19 years': 0.05,  # High school/early college
            '20 to 24 years': 0.60,  # Bachelor's graduates
            '25 to 34 years': 0.30,  # Master's/PhD students
            '35 to 44 years': 0.04,  # Career changers
            '45 to 54 years': 0.007,  # Rare cases
            '55 years and over': 0.003  # Very rare cases
        }
    },
    'International': {
        'total_pool': 500_000,  # Potential international candidates
        'distribution': {
            '16 to 19 years': 0.02,  # International students
            '20 to 24 years': 0.25,  # Fresh graduates
            '25 to 34 years': 0.40,  # Early career
            '35 to 44 years': 0.20,  # Mid career
            '45 to 54 years': 0.10,  # Senior level
            '55 years and over': 0.03  # Expert level
        }
    }
}

# Keep the qualification rates from before
hiring_channels = {
    'Industry': {
        '16 to 19 years': {'qualified_rate': 0.0},
        '20 to 24 years': {'qualified_rate': 0.15},
        '25 to 34 years': {'qualified_rate': 0.45},
        '35 to 44 years': {'qualified_rate': 0.60},
        '45 to 54 years': {'qualified_rate': 0.55},
        '55 years and over': {'qualified_rate': 0.40}
    },
    'Student': {
        '16 to 19 years': {'qualified_rate': 0.10},
        '20 to 24 years': {'qualified_rate': 0.35},
        '25 to 34 years': {'qualified_rate': 0.25},
        '35 to 44 years': {'qualified_rate': 0.05},
        '45 to 54 years': {'qualified_rate': 0.02},
        '55 years and over': {'qualified_rate': 0.01}
    },
    'International': {
        '16 to 19 years': {'qualified_rate': 0.05},
        '20 to 24 years': {'qualified_rate': 0.30},
        '25 to 34 years': {'qualified_rate': 0.50},
        '35 to 44 years': {'qualified_rate': 0.45},
        '45 to 54 years': {'qualified_rate': 0.35},
        '55 years and over': {'qualified_rate': 0.20}
    }
}
# Create a mapping between the two formats
age_group_mapping = {
    '16 to 19 years': '16-19',
    '20 to 24 years': '20-24',
    '25 to 34 years': '25-34',
    '35 to 44 years': '35-44',
    '45 to 54 years': '45-54',
    '55 years and over': '55+'
}

# Calculate feasible hires for each channel and age group
for channel, data in channel_population.items():
    print(f"\n{channel} Channel Feasible Hires:")
    print("Age Group | Base Pool | Unemployment | Qualified Rate | Feasible Hires")
    print("-" * 75)
    
    total_feasible = 0
    for age_group, distribution in data['distribution'].items():
        # Calculate base population for this age group
        base_pool = data['total_pool'] * distribution
        
        # Get predicted unemployment rate for 2024 using the mapped age group
        mapped_age_group = age_group_mapping[age_group]
        if mapped_age_group in predictions_by_age:
            pred_2024 = predictions_by_age[mapped_age_group].loc[
                predictions_by_age[mapped_age_group]['Date'].dt.year == 2024, 
                'Predicted_Rate'
            ].iloc[0] / 100  # Convert percentage to decimal
            
            # Calculate feasible hires
            qualified_rate = hiring_channels[channel][age_group]['qualified_rate']
            feasible_hires = base_pool * pred_2024 * qualified_rate
            total_feasible += feasible_hires
            
            print(f"{age_group:<20} | {base_pool:>9,.0f} | {pred_2024:>11.1%} | {qualified_rate:>13.1%} | {feasible_hires:>14,.0f}")
    
    print("-" * 75)
    print(f"Total Feasible Hires: {total_feasible:,.0f}")


Industry Channel Feasible Hires:
Age Group | Base Pool | Unemployment | Qualified Rate | Feasible Hires
---------------------------------------------------------------------------
16 to 19 years       |         0 |       10.5% |          0.0% |              0
20 to 24 years       |   100,000 |       11.1% |         15.0% |          1,666
25 to 34 years       |   350,000 |        9.6% |         45.0% |         15,163
35 to 44 years       |   300,000 |       10.2% |         60.0% |         18,337
45 to 54 years       |   150,000 |        9.1% |         55.0% |          7,475
---------------------------------------------------------------------------
Total Feasible Hires: 42,641

Student Channel Feasible Hires:
Age Group | Base Pool | Unemployment | Qualified Rate | Feasible Hires
---------------------------------------------------------------------------
16 to 19 years       |    10,000 |       10.5% |         10.0% |            105
20 to 24 years       |   120,000 |       11.1% |      

In [4]:

# Calculate feasible hires for each channel and age group by year
years = range(2024, 2034)
feasible_by_year_channel = {year: {} for year in years}

for year in years:
    print(f"\n=== Feasible Hires for {year} ===")
    
    for channel, data in channel_population.items():
        total_feasible = 0
        print(f"\n{channel} Channel:")
        print("Age Group | Base Pool | Unemployment | Qualified Rate | Feasible Hires")
        print("-" * 75)
        
        for age_group, distribution in data['distribution'].items():
            # Calculate base population for this age group
            base_pool = data['total_pool'] * distribution
            
            # Get predicted unemployment rate for the current year
            mapped_age_group = age_group_mapping[age_group]
            if mapped_age_group in predictions_by_age:
                pred_rate = predictions_by_age[mapped_age_group].loc[
                    predictions_by_age[mapped_age_group]['Date'].dt.year == year, 
                    'Predicted_Rate'
                ].iloc[0] / 100
                
                # Calculate feasible hires
                qualified_rate = hiring_channels[channel][age_group]['qualified_rate']
                feasible_hires = base_pool * pred_rate * qualified_rate
                total_feasible += feasible_hires
                
                print(f"{age_group:<20} | {base_pool:>9,.0f} | {pred_rate:>11.1%} | {qualified_rate:>13.1%} | {feasible_hires:>14,.0f}")
        
        print("-" * 75)
        print(f"Total Feasible Hires: {total_feasible:,.0f}")
        feasible_by_year_channel[year][channel] = total_feasible

# Create summary table
print("\n=== Summary: Feasible Hires by Year and Channel ===")
print("Year |", " | ".join(f"{channel:>12}" for channel in channel_population.keys()))
print("-" * 60)
for year in years:
    print(f"{year} |", " | ".join(f"{feasible_by_year_channel[year][channel]:>12,.0f}" for channel in channel_population.keys()))


=== Feasible Hires for 2024 ===

Industry Channel:
Age Group | Base Pool | Unemployment | Qualified Rate | Feasible Hires
---------------------------------------------------------------------------
16 to 19 years       |         0 |       10.5% |          0.0% |              0
20 to 24 years       |   100,000 |       11.1% |         15.0% |          1,666
25 to 34 years       |   350,000 |        9.6% |         45.0% |         15,163
35 to 44 years       |   300,000 |       10.2% |         60.0% |         18,337
45 to 54 years       |   150,000 |        9.1% |         55.0% |          7,475
---------------------------------------------------------------------------
Total Feasible Hires: 42,641

Student Channel:
Age Group | Base Pool | Unemployment | Qualified Rate | Feasible Hires
---------------------------------------------------------------------------
16 to 19 years       |    10,000 |       10.5% |         10.0% |            105
20 to 24 years       |   120,000 |       11.1% |   

In [5]:
def get_feasible_hires_by_year(channel_population, hiring_channels, predictions_by_age, age_group_mapping):
    """
    Calculate feasible hires for each year from 2024 to 2033
    
    Returns:
    DataFrame with columns: Year, Feasible_Hires
    """
    years = range(2024, 2034)
    feasible_hires = pd.DataFrame(index=years, columns=['Feasible_Hires'])
    
    for year in years:
        total_feasible = 0
        for channel, data in channel_population.items():
            for age_group, distribution in data['distribution'].items():
                base_pool = data['total_pool'] * distribution
                mapped_age_group = age_group_mapping[age_group]
                
                if mapped_age_group in predictions_by_age:
                    pred_rate = predictions_by_age[mapped_age_group].loc[
                        predictions_by_age[mapped_age_group]['Date'].dt.year == year, 
                        'Predicted_Rate'
                    ].iloc[0] / 100
                    
                    qualified_rate = hiring_channels[channel][age_group]['qualified_rate']
                    feasible_hires.loc[year, 'Feasible_Hires'] = base_pool * pred_rate * qualified_rate
                    total_feasible += feasible_hires.loc[year, 'Feasible_Hires']
        
        feasible_hires.loc[year, 'Feasible_Hires'] = total_feasible
    
    return feasible_hires